In [ ]:
import os
import flexynesis
import torch
import numpy as np
import pandas as pd

In [ ]:
cbio = flexynesis.CBioPortalData('msk_impact_2017')
cbio.get_cbioportal_data(cbio.study_id)

In [ ]:
cna = pd.read_csv('./msk_impact_2017/data_cna.txt', delimiter='\t')

In [ ]:
cna.head()

In [ ]:
mut = pd.read_csv('./msk_impact_2017/data_mutations.txt', delimiter='\t', skiprows=1)

In [ ]:
mut.head()

In [ ]:
clin = pd.read_csv('./msk_impact_2017/data_clinical_sample.txt', delimiter='\t', skiprows=4)

In [ ]:
clin.head()

In [ ]:
cbio.get_cbioportal_data(cbio.study_id, files={'mut': 'data_mutations.txt', 'cna': 'data_cna.txt', 'clin': 'data_clinical_sample.txt'})

In [ ]:
data = cbio.data

In [ ]:
data['clin'].index = data['clin'].SAMPLE_ID

In [ ]:
counts = np.unique(list(data['clin'].PRIMARY_SITE), return_counts=True) 

In [ ]:
counts

In [ ]:
cancer_types = counts[0][np.where(counts[1] > 300)]

In [ ]:
cancer_types

In [ ]:
clin_subset = data['clin'][data['clin']['PRIMARY_SITE'].isin(cancer_types)]

In [ ]:
data['clin'] = clin_subset

In [ ]:
cbio.data = data

In [ ]:
dat_split = cbio.split_data()

In [ ]:
{x: dat_split['test'][x].shape for x in dat_split['test'].keys()}

In [ ]:
cbio.print_dataset(dat_split, 'msk_impact')

In [ ]:
data_importer = flexynesis.DataImporter(path ='msk_impact/', 
                                       data_types = ['mut', 'cna'], 
                                        concatenate=False, top_percentile=90, variance_threshold=0.1,
                                        min_features=500)
train_dataset, test_dataset = data_importer.import_data()

In [ ]:
train_dataset.ann

In [ ]:
np.unique(train_dataset.ann['PRIMARY_SITE'], return_counts=True)

In [ ]:
train_dataset.label_mappings['PRIMARY_SITE']

In [ ]:
flexynesis.print_summary_stats(train_dataset)